In [1]:
import xlrd
import pandas as pd
from pandas import DataFrame

In [2]:
wbk = xlrd.open_workbook('DK_plastics_export.xlsx')
doc = wbk.sheet_by_index(0)
# Cells are identified by (row, col) where A1 is (0, 0)
# This spreadsheet uses only columns 0 and 2
# ...except for one chunk (942-951) that uses 0 and 1.

contents_start = 32 # Table of contents lists category & headings
contents_end = 177
subst_start = 689   # Tables of substance info begin
subst_end = 2365

In [3]:
items = dict()

In [4]:
def get_substance(r):
    d = dict()
    # There are 10 data elements per substance (BFRs has only 9)
    # plus an extra 'Substance' element if there is a page break.
    for i in range(r, r+12):
        a = str(doc.cell_value(i, 0)).strip()
        # Some quick cleanup of dict keys
        k = ' '.join([x for x in a.split() if x != 'the'][:3])
        k = k.replace('types of', 'plastics')
        b = str(doc.cell_value(i, 2)).strip() or str(doc.cell_value(i, 1)).strip()
        if a == 'References':
            d[k] = b
            break
        else:
            d[k] = b
    return d

In [5]:
# The main extraction of tables:
for r in range(subst_start, subst_end):
    h = str(doc.cell_value(r, 0)).strip().split(maxsplit=1)
    if len(h[0].split('.')) == 3: # look for headings that match x.x.x
        d = get_substance(r+1)
        items[' '.join(h)] = d

In [6]:
ordered_index = ['Substance',  'Function', 'Grouping', 'Main article groups', 
                 'Relevant plastics', 'CAS Number', 'Justification', 'Fate of substance',  
                 'Potential for exposure', 'Potential for release', 'References']
df = DataFrame(items, index=ordered_index)

In [7]:
groupings = {'3.11': 'Others'}

for r in range(contents_start, contents_end):
    h = str(doc.cell_value(r, 0)).strip().split(maxsplit=1)
    if len(h[0].split('.')) == 2:
        groupings[h[0]] = h[1].split('..', maxsplit=1)[0]

    # Show me:
DataFrame(groupings, index=['title'])

,3.10,3.11,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9
title,Solvents –neutral and reactive,Others,Antimicrobial substances,Blowing agents,"Heavy metal based colorants, stabilisers and c...",Flame retardants,"Monomers, cross linkers, hardeners, chain modi...",Organic based colorants,"UV stabilisers, antioxidants and other stabili...",Plasticisers


In [8]:
for j in df.columns:
    h = j.split(maxsplit=1)
    k = '.'.join(h[0].split('.')[:2])
    df[j]['Grouping'] = groupings[k]

for j in df.columns:
    h = j.split(maxsplit=1)
    df.rename_axis({j: h[1]}, axis=1, inplace=True)

In [9]:
df.head()

,2-Methoxyethanol,Trichloroethylene,"N,N-dimethylformamide (DMF)","1,2,3-Trichloropropane","1,6-hexanediol diglycidyl ether","Dichloromethane, methylene chloride","1,1,2-trichloroethane","1,4-dioxane",Nonylphenol and its ethoxylates,Octylphenol and its ethoxylates,...,"1,2-Benzenedicarboxylic acid, di-C6-8-branched alkyl esters, C7-rich",Glycerol triacetate (GTA),Trimethyl pentaryl diisobutyrate (TXIB),"Alkanes, C10-13, chloro (Short Chain Chlorinated Paraffins-SCCP)",Medium-chain chlorinated paraffins (MCCP),Bis(2-methoxyethyl) phthalate (DMEP),Benzyl butyl phthalate (BBP),Bis (2-ethylhexyl)phthalate (DEHP),Di (2-ethyl-hexyl) terephthalate (DEHT),Dibutyl phthalate (DBP)
Substance,2-Methoxyethanol,Trichloroethylene,"N,N-dimethylformamide (DMF)","1,2,3-Trichloropropane","1,6-hexanediol diglycidyl ether","Dichloromethane, methylene chloride","1,1,2-trichloroethane","1,4-dioxane",Nonylphenol and its etoxylates,Octylphenol and its ethoxylates,...,"1,2-Benzenedicarboxylic acid, di-C6-8-branched...",Glycerol triacetate (GTA),Trimethyl pentaryl diisobutyrate (TXIB),"Alkanes, C10-13, chloro (Short Chain Chlorinat...",Medium-chain chlorinated paraffins (MCCP),Bis(2-methoxyethyl) phthalate (DMEP),Benzyl butyl phthalate (BBP),Bis (2-ethylhexyl)phthalate (DEHP ),Di (2-ethyl-hexyl) terephthalate (DEHT),Dibutyl phthalate (DBP)
Function,"Solvent, chemical intermediate and solvent cou...",Intermediate or chain transfer agent for contr...,Chemical intermediate/solvent.,Intermediate - Crosslinking agent [Annex XV do...,"Reactive dilutant, stabiliser.",Intermediate.,Intermediate.,Intermediate.,Nonylphenol:\nMonomer (for phenol/formaldehyde...,Octylphenol:\nMonomer (for phenol formaldehyde...,...,Plasticiser [Annex XV dossier].,Plasticiser.,Plasticiser.,Secondary plasticiser and flame retardant [US ...,"Plasticiser and flame retardant [KLIF, 2010].",Plasticiser [Annex XV dossier].,Plasticiser [Annex XV].,Plasticiser [Annex XV dossier].,Plasticiser.,"Plasticiser [Annex XV dossier], component in c..."
Grouping,Solvents –neutral and reactive,Solvents –neutral and reactive,Solvents –neutral and reactive,Solvents –neutral and reactive,Solvents –neutral and reactive,Solvents –neutral and reactive,Solvents –neutral and reactive,Solvents –neutral and reactive,Others,Others,...,Plasticisers,Plasticisers,Plasticisers,Plasticisers,Plasticisers,Plasticisers,Plasticisers,Plasticisers,Plasticisers,Plasticisers
Main article groups,Not specified. Plastic products made of epoxy ...,Not specified - probably all product types.,Artificial fibres/artificial leather.,Tetrafluoroethylene-Hexafluoropropylene-Copoly...,No information.,No information.,Tubings etc.,No information.,No information.,OP-based resins are used for secondary insulat...,...,"Plasticiser in PVC: vinyl flooring, tile and c...","PVA-adhesives, coatings.","Toys, childcare products.",Polyurethane and acrylic sealants. Fire proofi...,Polyester e.g. in lifeboats.\nPVC - mainly wal...,Uses that have been reported include material ...,"PVC flooring (largest single use category, 41%...",DEHP has for many years been one of the domina...,DEHP-applications.,DBP has for many years been one of the dominan...
Relevant plastics,Epoxy resins and polyvinylacetate. The knowled...,PVC.,"polyacrylonitrile, polyurethane and polyvinylc...",Hexafluoropropylene [Annex XV dossier] which i...,"Epoxies, chlorinated vinyl resins.",Polycarbonate. Has been used as blowing agent ...,Teflon.,"Cellulose based polymers (cellulose acetate, e...","Phenol/formaldehyde plastic, epoxy [EU Commiss...","Phenol formaldehyde resin, PVC, styrene-butadi...",...,"PVC, one-component polyurethanes and acrylics ...",PVC etc.,PVC etc.,In USA use in PVC is important [US EPA 2009a]....,"Soft plastic, PVC, and polyester [KLIF, 2010].","Nitrocellulose, acetyl cellulose, polyvinyl ac...",BBP is or has been widely used in PVC (60% of ...,"PVC (mainly), but also other polymer products ...",PVC etc.,"PVC (plasticiser) [Annex XV dossier], PP (cata..."


In [10]:
dft = df.transpose()
dft.to_excel('plastic-additives-compiled.xlsx', sheet_name='Sheet1')